In [1]:
from docplex.cp.model import *
from docplex.cp.config import get_default

In [ ]:
H = ['f1', 'f2', 'f3', 'f4']
F = ['h1', 'h2', 'h3', 'h4']

L = {
    "f1":['h2', 'h1', 'h3', 'h4'],
    "f2":['h3', 'h4', 'h1', 'h2'],
    "f3":['h1', 'h3', 'h4', 'h2'],
    "f4":['h2', 'h1', 'h3', 'h4'],

    "h1":['f2', 'f3', 'f1', 'f4'],
    "h2":['f4', 'f1', 'f3', 'f2'],
    "h3":['f2', 'f4', 'f1', 'f3'],
    "h4":['f3', 'f1', 'f4', 'f2']
}

mdl = CpoModel()
